In [10]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [11]:
elo_hard=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)

elo_clay=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)


In [12]:
elo_data_hard =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19 --and Winner_Rank BETWEEN 1 AND 200 --and Winner_Rank<1000",
    con=devengine,)


elo_data_clay =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19",
    con=devengine,)

In [13]:
def get_player_record(player,opponent_rank,history,time,opponent,rank):
    player_history_top50=pd.DataFrame()
    player_history_top100=pd.DataFrame()
    player_history=pd.DataFrame()
    if opponent_rank<50:
        player_history_top50=history[(((history['Fav']==player)&(history['Dog_Rank']<50))|((history['Dog']==player)&(history['Fav_Rank']<50)))]
        #if player_history_top50.empty==False and (len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))  > 0.5:
            
            #print(time,player,"Top50",len(player_history_top50))
            #print(len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))        
    if opponent_rank<100:
        player_history_top100=history[(((history['Fav']==player)&(history['Dog_Rank']<100))|((history['Dog']==player)&(history['Fav_Rank']<100)))]
        #if player_history_top100.empty==False and (len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   > 0.5:     
        #    print(time,player,"Top100",len(player_history_top100))
        #    print(len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   
            
    else:
        opponent_rank_low=opponent_rank-50
        opponent_rank_high=opponent_rank+50
        player_history=history[((history['Fav']==player)&((history['Dog_Rank']>opponent_rank_low)&(history['Dog_Rank']<opponent_rank_high)))|
        ((history['Dog']==player)&((history['Fav_Rank']>opponent_rank_low)&(history['Fav_Rank']<opponent_rank_high)))]
        #if player_history.empty==False:
        #    if (len(player_history[player_history['Winner']==player])/len(player_history)) > 0.5 and len(player_history) >5:
        #        print(time,player,opponent_rank_low,opponent_rank_high,len(player_history))
        #        print(len(player_history[player_history['Winner']==player])/len(player_history))       
    
    return player_history,player_history_top100,player_history




In [14]:
def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


In [15]:
result_df=pd.DataFrame()
for _,row in elo_data_hard.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)       
        
    if True and games > 4 and games2 > 4:# and (((percent >= 0.7)&((percent2 < 0.5))) | ((percent2 >= 0.7)&((percent < 0.5)))):
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav": [row.Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
#result_df      


In [16]:
serving=pd.read_excel('servers_today.xlsx')
serving_womens=pd.read_excel('servers_today_womens.xlsx')
serving=pd.concat([serving,serving_womens])
serving=serving.drop(columns='Time')
result=pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])#.to_clipboard(index=False)
result#[(result['Fav_Serve%']>69)|(result['Dog_Serve%']>69)]

,Time,Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
0,06:15,August Holmgren,64%,11,Connor Farren,0%,5,29.0,0.0,0.0,0.0
1,06:35,Johannes Haerteis,60%,10,Evan King,40%,5,NaN,NaN,NaN,NaN
2,07:20,Dominik Koepfer,82%,11,Aleksandar Vukic,72%,18,73.0,13.0,82.0,22.0
3,07:55,Maks Kasnikowski,83%,6,Benjamin Sigouin,20%,5,NaN,NaN,NaN,NaN
4,10:00,Sasi Kumar Mukund,73%,11,Naoki Nakagawa,33%,6,NaN,NaN,NaN,NaN
5,10:00,Jason Jung,60%,10,Yuki Mochizuki,40%,5,74.0,0.0,0.0,18.0
6,11:30,Yunseong Chung,64%,11,Yuta Shimizu,40%,5,80.0,13.0,38.0,6.0
7,11:30,Shintaro Mochizuki,64%,11,Rimpei Kawakami,40%,5,50.0,0.0,0.0,20.0
8,13:00,Rinky Hijikata,62%,13,Dane Sweeny,40%,5,73.0,20.0,65.0,12.0
9,13:00,David Ionel Nicholas,50%,10,Nino Serdarusic,38%,8,NaN,9.0,80.0,NaN


In [17]:
result_df=pd.DataFrame()
for _,row in elo_data_clay.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_clay,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_clay,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_clay,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_clay,low_limit,high_limit,True)       
        
    if True and games > 4 and games2 > 4 and (((percent >= 0.7)&((percent2 < 0.5))) | ((percent2 >= 0.7)&((percent < 0.5)))):
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav": [row.Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
#result_df      


In [18]:
serving=pd.read_excel('servers_today.xlsx')
serving_womens=pd.read_excel('servers_today_womens.xlsx')
serving=pd.concat([serving,serving_womens])
serving=serving.drop(columns='Time')
result=pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])#.to_clipboard(index=False)
result[(result['Fav_Serve%']>69)|(result['Dog_Serve%']>69)]

,Time,Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
0,05:10,Tomas Martin Etcheverry,86%,35,Alberto Olivieri Genaro,20%,5,70.0,NaN,NaN,15.0
